# Récupération et traitement des données

In [119]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx
import pynsee as yns
import requests
from shapely.geometry import Point
from geopy.geocoders import Nominatim
from cartiflette.s3 import download_vectorfile_url_all

In [120]:
from pathlib import Path

In [121]:
Path.cwd()

PosixPath('/home/cathu/Documents/ENSAE/Projet_Catherine_Christelle')

## Les aménagements cyclables en Ile de France

In [122]:
a_velo= gpd.read_file('amenagements-velo-en-ile-de-france.geojson')

In [5]:
# Nombre d'entrées
a_velo.shape[0]

112217

In [6]:
a_velo.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Un préalable au calcul des surfaces est le choix du système de projection adéquat. Dans notre cas , il s'agit de convertir les données au système de projection Lambert 93 qui est le plus approprié.

In [7]:
a_velo['longueur'].describe()

count    112217.000000
mean        126.315959
std         174.519817
min           0.000000
25%          24.000000
50%          70.000000
75%         161.000000
max        3984.000000
Name: longueur, dtype: float64

In [8]:
a_velo.sample(n=1)

,osm_id,nom_com,sens_voit,ag,panneaux,moyenn_ech,revetement,highway,insee_com,nom_voie,longueur,petite_ech,nv,ad,geometry
100698,792297373.0,Mézières-sur-Seine,UNIQUE,None,None,22,asphalt,secondary,78402,Route de Paris à Cherbourg,23,2,None,bande uni,"LINESTRING (1.79807 48.96214, 1.79776 48.96217)"


Pour le moment, nous travaillons sans les données de géolocalisation qui ne sont pas nécessaires aux calculs. Nous créons pour cela une nouvelle dataframe

In [9]:
col_to_keep = ["nom_com", "sens_voit", "ag", "panneaux", "revetement", "highway", "insee_com", "longueur", "nv", "ad"] 

In [52]:
# df_a_velo = a_velo[col_to_keep].copy()

df_a_velo = a_velo

print(df_a_velo.head())

         osm_id            nom_com sens_voit                              ag  \
0  4.014620e+08            Chelles    DOUBLE                            None   
1  4.048666e+08   La Queue-en-Brie        NC  chemin service site propre uni   
2  1.154304e+09            Lésigny        NC                  voie verte uni   
3  3.300503e+08  Pontault-Combault    UNIQUE                       DSC bande   
4  1.104033e+09   Champs-sur-Marne    UNIQUE                            None   

  panneaux moyenn_ech revetement      highway insee_com  \
0     None         32       None      service     77108   
1     None         11       None        track     94060   
2     None         11    asphalt         path     77249   
3     None         22       None  residential     77373   
4     None         22    asphalt  residential     77083   

                     nom_voie  longueur petite_ech            nv  \
0  Rue de la Mare Longue Noue        24          3     limite 30   
1             Allée Jacquett

### Toutes les pistes ne se valent pas : création de variables d'étude pour les aménagements cyclables

Nous souhaitons étudier la répartition géograhique des aménagements cyclables et particulièrement leur densité en mètre par habitants. Toutefois, l'ensemble des pistes cyclables n'est pas de la même "qualité" : séparée de la route ou non, sens inverse de la circulation, type de revêtement, etc. Pour prendre en compte la qualité des aménagements cyclables, nous pouvons donner des coefficients aux mètres de pistes selon ces différents critères lors du calcul du nombre total de mètres aménagés par commune. Nous allons proposer plusieurs méthodes de calcul selon ces critères.

In [80]:
print(df_a_velo["sens_voit"].unique())
print(df_a_velo["ag"].unique())
print(df_a_velo["revetement"].unique())
print(df_a_velo["highway"].unique())
print(df_a_velo["nv"].unique())
print(df_a_velo["ad"].unique())

['DOUBLE' 'NC' 'UNIQUE']
[None 'chemin service site propre uni' 'voie verte uni' 'DSC bande'
 'autre chemin velo uni' 'piste uni' 'DSC' 'piste trottoir uni'
 'bande uni' 'cheminement trottoir uni' 'goulotte' 'chemin dedie uni'
 'chaucidou' 'cheminement uni' 'DSC piste' 'voie bus uni' 'piste bi'
 'bande bi' 'shoulder uni']
[None 'asphalt' 'compacted' 'concrete' 'unpaved' 'wood' 'paving_stones'
 'ground' 'sett' 'gravel' 'sand' 'paved' 'fine_gravel' 'dirt'
 'cobblestone' 'concrete:plates' 'concrete:lanes' 'grass'
 'cobblestone:flattened' 'metal' 'pebblestone' 'unhewn_cobblestone'
 'bricks' 'tartan' 'pavés' 'vegecol' 'artificial_turf' 'grass_paver'
 'earth' 'bitume' 'à_définir' 'rock']
['service' 'track' 'path' 'residential' 'footway' 'cycleway' 'secondary'
 'tertiary' 'living_street' 'unclassified' 'pedestrian' 'steps' 'primary'
 'primary_link' 'secondary_link' 'tertiary_link' 'trunk_link'
 'motorway_link']
['limite 30' 'hors voirie' 'unclassified' 'z20' 'rue pietonne' 'z30'
 'escalier ve

In [12]:
df_a_velo['sens_voit'].isna().sum()

0

In [13]:
df_a_velo['ag'].isna().sum()

68312

In [14]:
df_a_velo['revetement'].isna().sum()

38711

In [15]:
df_a_velo['highway'].isna().sum()

0

In [79]:
df_a_velo['nv'].isna().sum()

0

In [17]:
df_a_velo['ad'].isna().sum()

68522

In [75]:
print(df_a_velo['highway'].value_counts().get('unclassified', 0))

9548
4025


In [77]:
df_a_velo['adg'] = df_a_velo.apply(lambda row: row['ad'] if pd.notna(row['ad']) else (row['ag'] if pd.notna(row['ag']) else 'unclassified'), axis=1)

In [78]:
print(df_a_velo['adg'].value_counts().get('unclassified', 0))

55496


Seul le critère suivant ne présente pas trop de valeurs manquantes (nommées 'unclassified' dans la base) :  type de route ("highway", cf [documentation OpenstreeMap](https://wiki.openstreetmap.org/wiki/Key:highway) ). Les autres variables ont soit trop de valeurs manquantes soit manquent d'intérêt seule (sens des voitures). Cependant, "highway" est une colonne générale de catégorisation pour les données d'OpenStreetMap et manque de spécificité pour l'étude des pistes cyclables. Nous allons donc proposer deux méthodes de pondération (en plus d'une variable non pondérée) : une avec "highway", une avec "adg" (['nature de la voie'](https://opendata.stif.info/api/datasets/1.0/amenagements-velo-en-ile-de-france/attachments/metadonnees_amenagements_velo_en_ile_de_france_pdf/)), en proposant de normer les chemins dont nous n'avons pas connaissance de la qualité. Le nombre de valeurs manquantes est cependant très important, malgré notre tentative de réduire son ampleur en sommant ad et ag (ce sont les types de voies à gauche et à droite). Roulant à droite en, nous avons donner la priorité à voie à droite.

Nous allons produire 3 variables : deux pondérées censées prendre en compte la qualité de la route, et une autre sans pondération, pour la longueur d'aménagements cyclables par ville. Pour prendre en compte la qualité nous trions les types de routes en leur affectant des poids selon leurs caractéristiques : bande le long d'une route ou séparation, chemin avec ou sans voiture, avec ou sans piéton, etc. Pour cela, la variable "adg" nous semble plus adaptée car plus précise, mais elle a le défaut de présenter énormément valeurs manquantes. Lorsque la valeur est manquante nous assignons un poids neutre (1). La comparaison des résultats avec les 2 méthodes de pondération ou la méthode sans pondération pourra aussi nous renseigner sur la qualité de notre catégorisation.

In [81]:
print(df_a_velo["adg"].unique())

['unclassified' 'chemin service site propre uni' 'voie verte uni'
 'DSC bande' 'bande uni' 'autre chemin velo uni' 'piste uni' 'DSC'
 'piste trottoir uni' 'cheminement trottoir uni' 'goulotte'
 'chemin dedie uni' 'chaucidou' 'cheminement uni' 'voie bus uni'
 'DSC piste' 'piste bi' 'bande bi' 'shoulder uni']


In [99]:
highway_quality_mapping = {
    'service': 1,
    'track': 1,
    'path': 1,
    'trunk_link': 1,
    'motorway_link': 1,
    'residential': 2,
    'footway': 2,
    'cycleway': 2,
    'primary': 2,    
    'primary_link': 2,
    'unclassified': 2,
    'secondary': 3,
    'tertiary': 3,
    'secondary_link': 3,
    'tertiary_link': 3,
    'living_street': 3,
    'pedestrian': 4,
    'steps': 4
}


adg_quality_mapping = {
    'chemin service site propre uni': 1,
    'chemin dedie uni': 1,
    'voie verte uni': 1,
    'autre chemin velo uni': 1,
    'piste bi' : 1,
    'piste uni': 1,
    'bande uni': 2,
    'bande bi' : 2,
    'unclassified': 2,
    'cheminement trottoir uni' : 3,
    'piste trottoir uni': 3,
    'chaucidou': 3,
    'cheminement uni': 3,
    'shoulder uni' : 3,
    'voie bus uni': 4,
    'goulotte': 4,
    'DSC': 4,
    'DSC bande' : 4,
    'DSC piste' : 4
}

In [100]:
df_a_velo['qual_hw'] = df_a_velo['highway'].map(highway_quality_mapping)

df_a_velo['qual_adg'] = df_a_velo['adg'].map(adg_quality_mapping)

In [101]:
quality_weights = {
    1: 1.25,
    2: 1.,
    3: 0.75,
    4: 0.5
}

In [102]:
# Nouvelle colonne pour la longueur pondérée highway

df_a_velo['longueur_pond_hw'] = df_a_velo['longueur'] * df_a_velo['qual_hw'].map(quality_weights)

# Nouvelle colonne pour la longueur pondérée nature voie

df_a_velo['longueur_pond_adg'] = df_a_velo['longueur'] * df_a_velo['qual_adg'].map(quality_weights)

In [105]:
total_longueur_pond_hw = df_a_velo['longueur_pond_hw'].sum()
print(total_longueur_pond_hw) #longueur totale de l'échantillon avec majoration pondérée

14028145.5


In [106]:
total_longueur_pond_nv = df_a_velo['longueur_pond_adg'].sum()
print(total_longueur_pond_nv) #longueur totale de l'échantillon avec majoration pondérée

14382204.5


In [107]:
total_longueur = df_a_velo['longueur'].sum()
print(total_longueur)

14174798


In [108]:
total_longueur_commune = df_a_velo.groupby(['nom_com', 'insee_com'])['longueur'].sum().reset_index()

In [109]:
total_longueur_pond_hw_commune = df_a_velo.groupby(['nom_com', 'insee_com'])['longueur_pond_hw'].sum().reset_index()

In [110]:
total_longueur_pond_adg_commune = df_a_velo.groupby(['nom_com', 'insee_com'])['longueur_pond_adg'].sum().reset_index()

In [111]:
df_amenagements = pd.merge(total_longueur_commune, total_longueur_pond_hw_commune, on=['nom_com', 'insee_com'], suffixes=('_non_pond', '_pond_hw'))

In [112]:
df_amenagements = pd.merge(df_amenagements, total_longueur_pond_adg_commune, on=['nom_com', 'insee_com'], suffixes=(None, '_pond_adg'))

In [113]:
df_amenagements.sample(10)

,nom_com,insee_com,longueur,longueur_pond_hw,longueur_pond_adg
267,Dammarie-les-Lys,77152,30862,31861.25,33337.50
929,Villaines-sous-Bois,95660,3260,3084.00,3305.75
86,Beynes,78062,27886,27517.00,27924.50
138,Brunoy,91114,30180,29856.00,31480.50
602,Montlignon,95426,10993,11361.25,12168.50
713,Pringy,77378,785,904.00,942.25
784,Saint-Lambert,78561,6022,6116.50,6197.50
94,Boinvilliers,78072,1855,1909.75,1855.00
889,Ussy-sur-Marne,77478,146,182.50,182.50
588,Montcourt-Fromonville,77302,4734,5337.00,4711.00


In [114]:
df_amenagements['difference_hw'] = df_amenagements['longueur_pond_hw'] - df_amenagements['longueur']
df_amenagements['difference_adg'] = df_amenagements['longueur_pond_adg'] - df_amenagements['longueur']

Nous avons créé nos trois indicateurs par commune : longueur et longueur pondérée (adg et hw) par commune, mais nous avons perdu de l'information dans cette opération puisque nous perdons de ce fait les coordonnées exactes des aménagements. Cependant, pour l'étude de la densité, cela pourra aussi s'avérer utile. Surtout nous avons besoin de ces indicateurs pour le travail économétrique que nous souhaitons mener, où nous considérons alors les communes comme des individus, et la longueur des pistes cyclables comme nous variable d'intérêt. Cependant, avant de passer à la modélisation économétrique pour tenter d'expliquer le développement des aménagements dans les différentes communes, nous souhaitons représenter et analyser spatialement la répartition des aménagements, leur densité (par habitant), leur densité (par kilomètre) et notamment compléter ces analyses grâce à la disponibilité des données du Vélib.

## Données et ressources nécessaires pour l'étude spatiale

In [134]:
communes = download_vectorfile_url_all(
    crs = 4326,
    borders="COMMUNE_ARRONDISSEMENT",
    values = ["75","77","78","91", "92", "93", "94","95"],
    vectorfile_format="topojson",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=COMMUNE_ARRONDISSEMENT/crs=4326/DEPARTEMENT=75/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 40.1kiB [00:00, 1.24MiB/s]


https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=COMMUNE_ARRONDISSEMENT/crs=4326/DEPARTEMENT=77/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 1.57MiB [00:01, 1.10MiB/s]


https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=COMMUNE_ARRONDISSEMENT/crs=4326/DEPARTEMENT=78/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


...

Downloading: : 120kiB [00:00, 947kiB/s]


https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=COMMUNE_ARRONDISSEMENT/crs=4326/DEPARTEMENT=94/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 111kiB [00:00, 1.22MiB/s]


https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=COMMUNE_ARRONDISSEMENT/crs=4326/DEPARTEMENT=95/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 372kiB [00:00, 1.26MiB/s]


Un préalable au calcul des surfaces est le choix du système de projection adéquat. Dans notre cas , il s'agit de convertir les données au système de projection Lambert 93 qui est le plus approprié.

In [143]:
communes['surface'] = communes.to_crs(2154).area

In [144]:
communes.sort_values('surface', ascending = False)

,id,ID,NOM,NOM_M,INSEE_COM,STATUT,POPULATION,INSEE_CAN,INSEE_ARR,INSEE_DEP,INSEE_REG,SIREN_EPCI,source,INSEE_COG,geometry,surface
111,COMMUNE_0000000009739723,None,Fontainebleau,FONTAINEBLEAU,77186,Sous-préfecture,15696,07,4,77,11,200072346,IGN:EXPRESS-COG-CARTO-TERRITOIRE,77186,"POLYGON ((2.74740 48.48329, 2.74783 48.48327, ...",1.722864e+08
148,COMMUNE_0000002200276627,None,Saint-Germain-en-Laye,SAINT-GERMAIN-EN-LAYE,78551,Sous-préfecture,44806,16,3,78,11,200058519,IGN:EXPRESS-COG-CARTO-TERRITOIRE,78551,"POLYGON ((2.07077 48.94649, 2.07138 48.94676, ...",5.313401e+07
135,COMMUNE_0000002200276622,None,Chenoise-Cucharmoy,CHENOISE-CUCHARMOY,77109,Commune simple,1640,18,3,77,11,200037133,IGN:EXPRESS-COG-CARTO-TERRITOIRE,77109,"POLYGON ((3.13963 48.59977, 3.13951 48.59983, ...",4.874811e+07
229,COMMUNE_0000000009738401,None,Sonchamp,SONCHAMP,78601,Commune simple,1600,14,2,78,11,200073344,IGN:EXPRESS-COG-CARTO-TERRITOIRE,78601,"POLYGON ((1.83232 48.56349, 1.83260 48.56506, ...",4.634931e+07
152,COMMUNE_0000000009739738,None,Étampes,ETAMPES,91223,Sous-préfecture,25629,08,1,91,11,200017846,IGN:EXPRESS-COG-CARTO-TERRITOIRE,91223,"POLYGON ((2.23350 48.39672, 2.23293 48.39656, ...",4.596232e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,COMMUNE_0000000009736527,None,Saint-Mandé,SAINT-MANDE,94067,Commune simple,22377,23,2,94,11,200054781/200057941,IGN:EXPRESS-COG-CARTO-TERRITOIRE,94067,"POLYGON ((2.41900 48.84939, 2.41900 48.84935, ...",9.057590e+05
155,COMMUNE_0000000009734016,None,Gouzangrez,GOUZANGREZ,95282,Commune simple,160,16,3,95,11,200035970,IGN:EXPRESS-COG-CARTO-TERRITOIRE,95282,"POLYGON ((1.90588 49.11131, 1.90559 49.11143, ...",7.725902e+05
181,COMMUNE_0000000009735023,None,Margency,MARGENCY,95369,Commune simple,2846,15,2,95,11,200056380,IGN:EXPRESS-COG-CARTO-TERRITOIRE,95369,"POLYGON ((2.29014 48.99680, 2.28979 48.99688, ...",7.250545e+05
11,COMMUNE_0000000009736027,None,Le Pré-Saint-Gervais,LE PRE-SAINT-GERVAIS,93061,Commune simple,17351,15,1,93,11,200054781/200057875,IGN:EXPRESS-COG-CARTO-TERRITOIRE,93061,"POLYGON ((2.41151 48.88322, 2.41148 48.88315, ...",7.027441e+05


In [145]:
communes.rename(columns={'INSEE_COG': 'insee_com'}, inplace=True)

In [146]:
communes.sample(n=10)

,id,ID,NOM,NOM_M,INSEE_COM,STATUT,POPULATION,INSEE_CAN,INSEE_ARR,INSEE_DEP,INSEE_REG,SIREN_EPCI,source,insee_com,geometry,surface
357,COMMUNE_0000000009735982,None,Crécy-la-Chapelle,CRECY-LA-CHAPELLE,77142,Commune simple,4713,21,1,77,11,200090504,IGN:EXPRESS-COG-CARTO-TERRITOIRE,77142,"POLYGON ((2.95975 48.86247, 2.95987 48.86239, ...",1.545597e+07
128,COMMUNE_0000000009735544,None,Aulnay-sur-Mauldre,AULNAY-SUR-MAULDRE,78033,Commune simple,1142,01,1,78,11,200059889,IGN:EXPRESS-COG-CARTO-TERRITOIRE,78033,"POLYGON ((1.82898 48.92671, 1.82885 48.92688, ...",2.238844e+06
22,COMMUNE_0000000009739737,None,Ormoy-la-Rivière,ORMOY-LA-RIVIERE,91469,Commune simple,911,08,1,91,11,200017846,IGN:EXPRESS-COG-CARTO-TERRITOIRE,91469,"POLYGON ((2.20315 48.39544, 2.20289 48.39414, ...",1.047494e+07
398,COMMUNE_0000000009738342,None,Vieux-Champagne,VIEUX-CHAMPAGNE,77496,Commune simple,189,14,3,77,11,247700701,IGN:EXPRESS-COG-CARTO-TERRITOIRE,77496,"POLYGON ((3.16787 48.57744, 3.16756 48.57745, ...",8.911260e+06
279,COMMUNE_0000000009737914,None,Ozouer-le-Voulgis,OZOUER-LE-VOULGIS,77352,Commune simple,1921,08,2,77,11,200070779,IGN:EXPRESS-COG-CARTO-TERRITOIRE,77352,"POLYGON ((2.80024 48.68516, 2.80037 48.68493, ...",1.135592e+07
193,COMMUNE_0000000009735543,None,Bazemont,BAZEMONT,78049,Commune simple,1649,01,3,78,11,200034130,IGN:EXPRESS-COG-CARTO-TERRITOIRE,78049,"POLYGON ((1.88315 48.91710, 1.88312 48.91710, ...",6.709576e+06
138,COMMUNE_0000000009736562,None,Villepreux,VILLEPREUX,78674,Commune simple,11003,15,4,78,11,200058782,IGN:EXPRESS-COG-CARTO-TERRITOIRE,78674,"POLYGON ((2.04170 48.84300, 2.03866 48.84139, ...",1.046625e+07
70,COMMUNE_0000000009737955,None,Briis-sous-Forges,BRIIS-SOUS-FORGES,91111,Commune simple,3342,05,3,91,11,249100074,IGN:EXPRESS-COG-CARTO-TERRITOIRE,91111,"POLYGON ((2.11967 48.65196, 2.11985 48.65201, ...",1.116746e+07
3,COMMUNE_0000000009735498,None,Livry-Gargan,LIVRY-GARGAN,93046,Commune simple,45692,11,2,93,11,200054781/200058790,IGN:EXPRESS-COG-CARTO-TERRITOIRE,93046,"POLYGON ((2.56443 48.92809, 2.56426 48.92786, ...",7.372933e+06
179,COMMUNE_0000000009737910,None,Beauvoir,BEAUVOIR,77029,Commune simple,192,14,2,77,11,200070779,IGN:EXPRESS-COG-CARTO-TERRITOIRE,77029,"POLYGON ((2.84873 48.65522, 2.84875 48.65521, ...",3.957711e+06


## Les vélibs en Ile de France

In [115]:
url2 = "https://opendata.paris.fr/explore/dataset/velib-emplacement-des-stations/download/?format=geojson&timezone=Europe/Berlin&lang=fr"

In [116]:
velib = gpd.read_file(url2)

In [123]:
velib.sample(n=15)

,capacity,name,stationcode,geometry
1093,50,Dom-Pérignon - Gravelle,12119,POINT (2.40988 48.82548)
28,43,Boulets - Faubourg Saint-Antoine,11010,POINT (2.39175 48.84926)
1123,34,Sommerard - Saint-Jacques,5002,POINT (2.34502 48.85031)
38,33,Saint-Fargeau - Mortier,20117,POINT (2.40791 48.87267)
857,30,Quai de l'Oise - Aisne,19128,POINT (2.38337 48.89062)
735,30,Gare de Robinson,23303,POINT (2.28183 48.77993)
990,33,Vistule - Choisy,13113,POINT (2.36157 48.82366)
98,30,Gare RER de Gentilly,42504,POINT (2.34103 48.81428)
815,33,Rossini - Laffitte,9022,POINT (2.33798 48.87335)
156,17,Galilée - Vernet,8003,POINT (2.29854 48.87172)


In [129]:
gdf_velib = velib

In [132]:
gdf_velib.sample(10)

,capacity,name,stationcode,geometry
974,39,Jussieu - Fossés Saint-Bernard,5021,POINT (2.35349 48.84719)
63,17,Square Pierre Lazareff,2004,POINT (2.34839 48.86714)
1204,60,Bois de Vincennes.,12041,POINT (2.41322 48.83381)
702,26,Mahatma Gandhi,16123,POINT (2.26421 48.87636)
61,20,Joffre - Aigle,22503,POINT (2.24510 48.90403)
727,36,Perle - Vieille du Temple,3008,POINT (2.36112 48.86013)
849,51,Gare de Lyon - Van Gogh,12006,POINT (2.37045 48.84384)
1381,24,Jules Guesdes - Général de Gaulle,21315,POINT (2.27893 48.81931)
1348,35,Marignan - Champs-Elysées,8013,POINT (2.30661 48.86950)
150,29,George Sand - Jean de La Fontaine,16116,POINT (2.26826 48.84987)


On associe désormais aux stations vélib leur localisation (commune) pour créer deux nouvelles variables d'intérêt : la capacité vélib par commune (cap_com) et le nombre de stations vélib par communes (stat_com)

In [148]:
# Étape 1 : Spatial join pour associer à chaque station Vélib le code commune et le nom de la commune
gdf_velib_with_communes = gpd.sjoin(gdf_velib, communes[['id', 'NOM', 'INSEE_COM', 'geometry']], predicate='within')

In [151]:
# Étape 2 : Créer les variables cap_com et stat_com
df_aggregated = gdf_velib_with_communes.groupby('INSEE_COM').agg(
    cap_com=('capacity', 'sum'),
    stat_com=('index_right', 'count')
).reset_index()

In [162]:
# Étape 3 : Merge avec le GeoDataFrame des communes
df_velib = pd.merge(communes, df_aggregated, how='left', on='INSEE_COM')

In [163]:
# Liste des colonnes à supprimer
colonnes_a_supprimer = ['ID', 'id','INSEE_CAN', 'INSEE_ARR', 'SIREN_EPCI', 'source', 'insee_com']

# Supprimer les colonnes
df_velib = df_velib.drop(colonnes_a_supprimer, axis=1)

In [164]:
df_velib.head(5)

,NOM,NOM_M,INSEE_COM,STATUT,POPULATION,INSEE_DEP,INSEE_REG,geometry,surface,cap_com,stat_com
0,Paris 3e Arrondissement,PARIS 3E ARRONDISSEMENT,75056,Capitale d'état,34025,75,11,"POLYGON ((2.35016 48.86199, 2.35019 48.86203, ...",1.170282e+06,32033.0,998.0
1,Paris 2e Arrondissement,PARIS 2E ARRONDISSEMENT,75056,Capitale d'état,21595,75,11,"POLYGON ((2.34792 48.87069, 2.34827 48.87062, ...",9.923087e+05,32033.0,998.0
2,Paris 4e Arrondissement,PARIS 4E ARRONDISSEMENT,75056,Capitale d'état,29131,75,11,"POLYGON ((2.36849 48.85580, 2.36873 48.85482, ...",1.596122e+06,32033.0,998.0
3,Paris 5e Arrondissement,PARIS 5E ARRONDISSEMENT,75056,Capitale d'état,58227,75,11,"POLYGON ((2.33666 48.83967, 2.33672 48.84011, ...",2.543140e+06,32033.0,998.0
4,Paris 6e Arrondissement,PARIS 6E ARRONDISSEMENT,75056,Capitale d'état,40303,75,11,"POLYGON ((2.33292 48.85934, 2.33339 48.85924, ...",2.152364e+06,32033.0,998.0


## Concaténation des bases d'aménagements cyclables

In [166]:
print(df_velib.columns)

Index(['NOM', 'NOM_M', 'INSEE_COM', 'STATUT', 'POPULATION', 'INSEE_DEP',
       'INSEE_REG', 'geometry', 'surface', 'cap_com', 'stat_com'],
      dtype='object')


In [167]:
print(df_amenagements.columns)

Index(['nom_com', 'insee_com', 'longueur', 'longueur_pond_hw',
       'longueur_pond_adg', 'difference_hw', 'difference_adg'],
      dtype='object')


In [169]:
# Colonnes à utiliser pour la fusion
colonnes_fusion = ['NOM', 'INSEE_COM']

# Fusion des DataFrames
df_velo = pd.merge(df_velib, df_amenagements, left_on=colonnes_fusion, right_on=['nom_com', 'insee_com'], how='outer')

In [170]:
df_velo.sample(15)

,NOM,NOM_M,INSEE_COM,STATUT,POPULATION,INSEE_DEP,INSEE_REG,geometry,surface,cap_com,stat_com,nom_com,insee_com,longueur,longueur_pond_hw,longueur_pond_adg,difference_hw,difference_adg
1255,Mours,MOURS,95436,Commune simple,1629.0,95,11,"POLYGON ((2.27931 49.12574, 2.27932 49.12571, ...",2.481698e+06,NaN,NaN,Mours,95436,1054.0,1001.75,1031.50,-52.25,-22.50
1159,Frouville,FROUVILLE,95258,Commune simple,349.0,95,11,"POLYGON ((2.12907 49.15387, 2.12949 49.15392, ...",7.474994e+06,NaN,NaN,Frouville,95258,6012.0,5569.75,6012.00,-442.25,0.00
623,Boissy-sans-Avoir,BOISSY-SANS-AVOIR,78084,Commune simple,653.0,78,11,"POLYGON ((1.79922 48.83137, 1.80008 48.83118, ...",4.074367e+06,NaN,NaN,Boissy-sans-Avoir,78084,810.0,793.75,810.00,-16.25,0.00
1298,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,20e Arrondissement,75120,112161.0,104438.00,90906.25,-7723.00,-21254.75
766,Grosrouvre,GROSROUVRE,78289,Commune simple,900.0,78,11,"POLYGON ((1.72683 48.78652, 1.72514 48.78937, ...",1.256439e+07,NaN,NaN,Grosrouvre,78289,487.0,365.25,487.00,-121.75,0.00
709,Conflans-Sainte-Honorine,CONFLANS-SAINTE-HONORINE,78172,Commune simple,35536.0,78,11,"POLYGON ((2.07192 48.99335, 2.07174 48.99364, ...",9.885484e+06,NaN,NaN,Conflans-Sainte-Honorine,78172,33696.0,31761.75,32990.75,-1934.25,-705.25
234,Fresnes-sur-Marne,FRESNES-SUR-MARNE,77196,Commune simple,971.0,77,11,"POLYGON ((2.75078 48.92343, 2.75052 48.92350, ...",7.487555e+06,NaN,NaN,Fresnes-sur-Marne,77196,1678.0,1678.00,2097.50,0.00,419.50
1280,Villeron,VILLERON,95675,Commune simple,1506.0,95,11,"POLYGON ((2.54995 49.05321, 2.54977 49.05322, ...",5.627301e+06,NaN,NaN,Villeron,95675,608.0,505.00,665.00,-103.00,57.00
595,Bouafle,BOUAFLE,78090,Commune simple,2194.0,78,11,"POLYGON ((1.88490 48.94046, 1.88459 48.94164, ...",6.962755e+06,NaN,NaN,Bouafle,78090,14898.0,14709.75,15163.50,-188.25,265.50
586,Carrières-sous-Poissy,CARRIERES-SOUS-POISSY,78123,Commune simple,16641.0,78,11,"POLYGON ((2.04931 48.95873, 2.04944 48.95766, ...",7.275577e+06,NaN,NaN,Carrières-sous-Poissy,78123,28058.0,27376.25,30415.25,-681.75,2357.25


In [171]:
# On supprime les communes pour lesquelles toutes les variables d'intérêt sont manquantes

# Liste des variables d'intérêt
variables_interet = ['longueur', 'longueur_pond_hw', 'longueur_pond_adg', 'difference_hw', 'difference_adg', 'cap_com', 'stat_com']

# Supprimer les lignes où toutes les variables d'intérêt sont manquantes
df_velo = df_velo.dropna(subset=variables_interet, how='all')


In [174]:
print(df_velo.shape)

(1026, 18)


Point d'étape : nous enregistrons cette base intermédiaire des vélos pour ne pas avoir à refaire toutes les étapes toujours.

In [ ]:
# Sauvegarder le DataFrame au format GeoJSON
df_velo.to_file('df_velo.geojson', driver='GeoJSON')


## Données socio-démographiques sur l'Ile de France

## Données complémentaires : accidents de la route, transports écologiques alternatifs (réseau ferré, bus)

## Base finale avec la longueur, les indicateurs géographiques, et les variables socio-démographiques

In [ ]:
base = communes[['insee_com', 'POPULATION', 'surface', 'geometry']].merge(y, how='outer', on='insee_com')

In [ ]:
base.groupby('insee_com').sum(numeric_only = True).sort_values('longueur', ascending = False)

In [ ]:
base = gpd.GeoDataFrame(base, geometry='geometry')

# Analyse descriptive et spatiale

## Répartition spatiale des aménagements cyclables : piste cyclable et station vélib

In [ ]:
# Répartition des stations vélib

fig,ax = plt.subplots(figsize=(10, 10))
velib.plot(ax = ax, color = 'green')
com.plot(ax = ax, zorder=1, edgecolor = "black", facecolor="none")
ax.set_axis_off()

In [ ]:
# Emplacement des pistes cycables (point)

fig,ax = plt.subplots(figsize=(10, 10))
a_velo.plot(ax = ax, color = 'red', alpha = 0.4, zorder=2)
communes.plot(ax = ax, zorder=1, edgecolor = "black", facecolor="none")
ax.set_axis_off()

In [ ]:
# Répartition des pistes cyclabes en île de France

fig, ax = plt.subplots(figsize=(10, 10))
dissolved = base.dissolve(by='insee_com', aggfunc='sum').reset_index()
dissolved.plot(ax=ax, column="longueur", legend=True)
ax.set_axis_off()
legend = ax.get_legend()
plt.show()

### Statistiques descriptives sur les pistes cyclables

In [ ]:
df_amenagements.describe()

In [ ]:
sns.scatterplot(df_amenagements['longueur'])
plt.title('Distribution des pistes cyclables dans les communes')